<a href="https://colab.research.google.com/github/mk0653/untitled/blob/master/Accident%20type%20classification/Accident_type_classification_pretrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi

Sat Oct 16 09:55:59 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install sentencepiece

     |████████████████████████████████| 1.2 MB 8.1 MB/s 


In [ ]:
!pip install transformers

     |████████████████████████████████| 2.9 MB 7.0 MB/s 
     |████████████████████████████████| 895 kB 68.5 MB/s 
     |████████████████████████████████| 596 kB 58.5 MB/s 
     |████████████████████████████████| 56 kB 6.3 MB/s 
     |████████████████████████████████| 3.3 MB 55.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

from transformers import (AutoModel,AutoModelForMaskedLM, 
                          AutoTokenizer, LineByLineTextDataset,
                          DataCollatorForLanguageModeling,
                          Trainer, TrainingArguments)

In [ ]:
import re
import os
import urllib.request
import unicodedata
import nltk

from bs4 import BeautifulSoup
from collections import Counter
from gensim import corpora
from nltk.corpus import wordnet
from tqdm.autonotebook import tqdm

#記号の除去
def clean_text(text):
    replaced_text = text.lower()
    replaced_text = re.sub(r'[【】]', ' ', replaced_text)       # 【】の除去
    replaced_text = re.sub(r'[（）()]', ' ', replaced_text)     # （）の除去
    replaced_text = re.sub(r'[［］\[\]]', ' ', replaced_text)   # ［］の除去
    replaced_text = re.sub(r'[@＠]\w+', '', replaced_text)  # メンションの除去
    replaced_text = re.sub(
        r'https?:\/\/.*?[\r\n ]', '', replaced_text)  # URLの除去
    replaced_text = re.sub(r'　', ' ', replaced_text)  # 全角空白の除去
    return replaced_text

In [ ]:
train_data = pd.read_csv('/content/drive/MyDrive/事故の型分類/sibou_db_h30_15_label.csv')
add_train_data = pd.read_csv('/content/drive/MyDrive/事故の型分類/sibou_db_h14-5_label.csv')
#add_train_data2 = pd.read_csv('/content/drive/MyDrive/事故の型分類/sisyou_db_h29_01-12_label.csv')

data = pd.concat([train_data,add_train_data])
train_texts = data["sentence"].tolist()

train_texts_clean = []
for x in tqdm(range(len(train_texts))):
  train_texts_clean.append(clean_text(train_texts[x]))

data = train_texts_clean
text  = '\n'.join(data)
with open('text.txt','w') as f:
    f.write(text)

  0%|          | 0/37637 [00:00<?, ?it/s]

In [ ]:
!pip install fugashi[unidic-lite]
!pip install ipadic

     |████████████████████████████████| 490 kB 10.0 MB/s 
     |████████████████████████████████| 47.4 MB 88 kB/s 
  Created wheel for unidic-lite: filename=unidic_lite-1.0.8-py3-none-any.whl size=47658836 sha256=d47512447687fb54cb1de74f83d6da051efded921cd24bf2a7237bcc0963ec95
  Stored in directory: /root/.cache/pip/wheels/de/69/b1/112140b599f2b13f609d485a99e357ba68df194d2079c5b1a2
Successfully built unidic-lite
     |████████████████████████████████| 13.4 MB 8.2 MB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556723 sha256=a3ca5620fcece54cb22000058d7a0afad41f7ee9a9f303e88e536eb890f42378
  Stored in directory: /root/.cache/pip/wheels/33/8b/99/cf0d27191876637cd3639a560f93aa982d7855ce826c94348b
Successfully built ipadic


In [ ]:
#model_name = "ALINEAR/albert-japanese-v2"
#model_name = "rinna/japanese-roberta-base"
model_name = "cl-tohoku/bert-large-japanese"

model = AutoModelForMaskedLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.save_pretrained('/content/drive/MyDrive/bert-large');

Downloading:   0%|          | 0.00/518 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Some weights of the model checkpoint at cl-tohoku/bert-large-japanese were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/174 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/230k [00:00<?, ?B/s]

In [ ]:
train_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="text.txt", #mention train text file here
    block_size=256)

valid_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="text.txt", #mention valid text file here
    block_size=256)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/bert-large", #select model path for checkpoint
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy= 'steps',
    save_total_limit=2,
    eval_steps=2000,
    save_steps= 2000,
    logging_steps =2000,
    metric_for_best_model='eval_loss',
    greater_is_better=False,
    load_best_model_at_end =True,
    prediction_loss_only=True,
    report_to = "none")

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset)

In [ ]:
trainer.train()
trainer.save_model(f'/content/drive/MyDrive/bert-large')

***** Running training *****
  Num examples = 37663
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 14124


Step,Training Loss,Validation Loss
2000,1.413300,1.216065
4000,1.245800,1.109140
6000,1.138100,1.013524
8000,1.067600,0.944949
10000,0.995500,0.892672
12000,0.928300,0.844384
14000,0.887500,0.814495


***** Running Evaluation *****
  Num examples = 37663
  Batch size = 8
Saving model checkpoint to /content/drive/MyDrive/bert-large/checkpoint-2000
Configuration saved in /content/drive/MyDrive/bert-large/checkpoint-2000/config.json
Model weights saved in /content/drive/MyDrive/bert-large/checkpoint-2000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 37663
  Batch size = 8
Saving model checkpoint to /content/drive/MyDrive/bert-large/checkpoint-4000
Configuration saved in /content/drive/MyDrive/bert-large/checkpoint-4000/config.json
Model weights saved in /content/drive/MyDrive/bert-large/checkpoint-4000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 37663
  Batch size = 8
Saving model checkpoint to /content/drive/MyDrive/bert-large/checkpoint-6000
Configuration saved in /content/drive/MyDrive/bert-large/checkpoint-6000/config.json
Model weights saved in /content/drive/MyDrive/bert-large/checkpoint-6000/pytorch_model.bin
Deleting older checkpoint [/

roberta base のスコア

Step	Training Loss	Validation Loss	Runtime	Samples Per Second

200	No log	1.441347	32.183100	88.276000

400	No log	1.383025	32.088000	88.538000

600	1.550100	1.340914	32.012800	88.746000

800	1.550100	1.319844	32.065800	88.599000